In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests2

In [2]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    x = np.random.uniform(low=-3., high=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs

yn,xn,nobs = gen_data()

In [3]:
def compute_loglike(resid):
    sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
    ll = np.log(stats.norm.pdf(resid,loc=0,scale=sigma))
    return ll

In [4]:
def compute_score(yn,xn,params):
    
    xn = sm.add_constant(xn)
    resid = yn - np.matmul(xn,params)
    
    k = len(params)
    scale = (resid**2).mean()
    
    tile_resid = np.tile( resid, k)
    tile_resid = np.reshape(tile_resid, (k,xn.shape[0]) ).transpose()
    grad = tile_resid*xn/scale     
    return grad
    
def compute_hess(yn,xn,params):
    pass


def setup_shi(yn,xn):
    x1n,x2n = xn[:,0:2],xn[:,1:3]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    k1 = len(model1_fit.params)
    
    ll1 = compute_loglike(model1_fit.resid)
    grad1 =  compute_score(yn,x1n,model1_fit.params)
    hess1 = model1.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    k2 = len(model1_fit.params)
    
    ll2 = compute_loglike(model2_fit.resid)
    grad2 =  compute_score(yn,x2n,model2_fit.params)
    hess2 = model2.hessian(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

# Main examples

In [5]:
#generate the sample
def gen_data(beta=1):
    nobs = 500
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 2*x[:,2] + 2*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=500,c=3)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(1000,setup_shi,gen_data)
print(shi_result)

reg: [0.95  0.022 0.028], boot1: [0.946 0.021 0.033], boot2: [0.946 0.021 0.033], llr:-0.4805492601816843, std: 22.16442878640675, omega:0.9973000519921233
[0.972 0.012 0.016]


In [6]:
#generate the sample
def gen_data(beta=3):
    nobs = 500
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(1000,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.993 0.005 0.002], boot2: [0.993 0.005 0.002], llr:0.16784420400880187, std: 1.2014428477172345, omega:0.061783095643456444
[1. 0. 0.]


# Other examples 

In [7]:
#generate the sample
def gen_data(beta=1):
    nobs = 250
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 2*x[:,2] + 2*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=500,c=10)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.95 0.01 0.04], boot1: [0.94 0.02 0.04], boot2: [0.94 0.02 0.04], llr:-2.1035723764218592, std: 15.907869209266984, omega:0.9982548572563921
[0.95 0.01 0.04]


In [8]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 2*x[:,2] + 2*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.946 0.025 0.029], boot1: [0.938 0.029 0.033], boot2: [0.938 0.029 0.033], llr:1.079463852114208, std: 31.246925559915244, omega:0.9976038608421213
[0.96 0.   0.04]


In [9]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 1*x[:,2] + 1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs

reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.95 0.03 0.02], boot1: [0.95 0.03 0.02], boot2: [0.95 0.03 0.02], llr:2.158888495170042, std: 32.972490786287, omega:0.9926272999842125
[0.94 0.02 0.04]


In [10]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.997 0.    0.003], boot2: [0.997 0.    0.003], llr:-0.027609454543062843, std: 0.9135450829069639, omega:0.038968966040388235
[1. 0. 0.]


In [11]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:43.48549630024081, std: 9.204253531705588, omega:0.2898659097890787
[0. 1. 0.]


In [12]:
#generate the sample
def gen_data(beta=2):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [1. 0. 0.], boot2: [1. 0. 0.], llr:0.25332673713984905, std: 1.39656037004952, omega:0.04931013977854134
[1. 0. 0.]
